<a href="https://colab.research.google.com/github/tarjerw/TDT4173-project-group6/blob/main/ML_project_(BOW).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Affiliation analysis of political tweets - Logistic regression

This is the code used for training a logistic regression method to do political affiliation classification (democrat or republican) of tweets, and testing it on test data, discussed in the paper Affiliation analysis of politcal tweets. Logistic regression is disscussed in section *4.1 Logistic regression in using bag-of-words(BOW)* and *5.1 Logistic regression method results*


## Import Libraries

The main libraries imported are numpy, pandas, matplotlib, sklearn.

In [ ]:
#import libraries for data manipulation and analysis   
import pandas as pd
import numpy as np

#plotting results
import matplotlib.pyplot as plt

#import the machine learning library: sklearn
import sklearn

#import bag of words of n-gram function (Countvectorizer) from sklearn
from sklearn.feature_extraction.text import CountVectorizer

#import function that seperates a dataset into a training set and test set with proportions specified in parameters
from sklearn.model_selection import train_test_split

# import the machine learning method, Logistic Regression used to classify tweets
from sklearn.linear_model import LogisticRegression

# import cross validation score function that computes k-folds cross validation with k specified in parameters
from sklearn.model_selection import cross_val_score

# import confusion matrix function from sklearn 
from sklearn.metrics import confusion_matrix

# TableIt is an efficient 3rd party file for displaying resuls in table form in python. 
# This file must be added to Collab before running.
import TableIt

##Data retrieval

Data is retrieved from the 2 CSV files TweetDatabase_2016.csv, TweetDatabase_2020.csv that contain all tweets from 2016 and 2020 respectively.
The files are first converted into pandas dataframes: df_2016, df_2020. Then they are merged together, and return a third dataset with both election years: df_2016_2020.

In [ ]:
#Retrieve data from CSV files and convert to dataframes using pandas
tweets2016_df = pd.read_csv('TweetDatabase_2016.csv')
tweets2020_df = pd.read_csv('TweetDatabase_2020.csv')
df_2016 = tweets2016_df 
df_2020 = tweets2020_df 

#All duplicates are removed from the dataframes, using .drop_duplicates()
df_2016 = df_2016.drop_duplicates()
df_2020 = df_2020.drop_duplicates() 
df_2016_2020 = df_2016.append(df_2020)

#Shuffle data using the pandas function .samples()
df_2016 = df_2016.sample(frac = 1)
df_2020 = df_2020.sample(frac = 1)
df_2016_2020 = df_2016_2020.sample(frac = 1)

##Sentiment analysis using logistic regression with BOW & N-Gram


###Splitting data

The 3 datasets are split into two mutually exclusive and collectively exhaustive classes: 80% of the original datasets are labeled as training sets and the remaining 20% are labeled as test sets using the sklearn method train_test_split()

In [ ]:
# Splits dataset into Training data and Test data, with 20% test data and 80% training data
# In sklearn's train_test_split() method, the argument test_size=0.2 specifies that 20% of the data set should be labeled test data and the remaining 80% should be labeled training data.

X_train16, X_test16, y_train16, y_test16 = train_test_split(df_2016.Tweet, df_2016.Party, test_size=0.2)
X_train20, X_test20, y_train20, y_test20 = train_test_split(df_2020.Tweet, df_2020.Party, test_size=0.2)
X_train1620, X_test1620, y_train1620, y_test1620 = train_test_split(df_2016_2020.Tweet, df_2016_2020.Party, test_size=0.2)

# Prints the resulting training and test sets for all three datasets.
print("Year 2016: \n"+"Democratic republican ratio in Training data:\n"+str(y_train16.value_counts())+"\n")
print("Democratic republican ratio in Test data:\n"+str(y_test16.value_counts()),'\n')
print("-------------------------------------------------------\n")
print("Year 2020: \n"+"Democratic republican ratio in Training data:\n"+str(y_train20.value_counts())+"\n")
print("Democratic republican ratio in Test data:\n"+str(y_test20.value_counts()),'\n')
print("-------------------------------------------------------\n")
print("Year 2016+2020: \n"+"Democratic republican ratio in Training data:\n"+str(y_train1620.value_counts())+"\n")
print("Democratic republican ratio in Test data:\n"+str(y_test1620.value_counts()))


###Transforming data

####Creating BOW model

In [ ]:
# Create Bag of words for each election year and preprocess data simultaneously using sklearns CountVectorizer() method.
# Parameters:
# max_df removes all words that are too frequent in the dataset 
# min_df removes all words that occur only once in the dataset 
# stop_words='english' removes most common, irrelevant english words
BagOfWords_vectorizer16 = CountVectorizer(lowercase=True, max_df=0.35, min_df=2, stop_words='english')
BagOfWords_vectorizer20 = CountVectorizer(lowercase=True, max_df=0.35, min_df=2, stop_words='english')
BagOfWords_vectorizer1620 = CountVectorizer(lowercase=True, max_df=0.35, min_df=2, stop_words='english')

# Learn the vocabulary of the training data and convert the training data into a document term matrix
# Parameters:
# fit(train_data) = learn the vocabulary of the training data
# transform(train_data) = convert the training data into a document term matrix
# fit_transform(train_data) = fit(train_data) + transform(train_data)
# transform(test_data) = use the fitted vocabulary (training) to build a document term matrix from the testing data

# 2016 dataset
X_train_BagOfWords16 = BagOfWords_vectorizer16.fit_transform(X_train16) 
X_test_BagOfWords16 = BagOfWords_vectorizer16.transform(X_test16)

# 2020 dataset
X_train_BagOfWords20 = BagOfWords_vectorizer20.fit_transform(X_train20)
X_test_BagOfWords20 = BagOfWords_vectorizer20.transform(X_test20)

# 20216-20 dataset
X_train_BagOfWords1620 = BagOfWords_vectorizer1620.fit_transform(X_train1620)
X_test_BagOfWords1620 = BagOfWords_vectorizer1620.transform(X_test1620)

####Creating N-Gram model 

In [ ]:
# Create N-Gram for each election year and preprocess data simultaneously using sklearns CountVectorizer() method.
# Parameters:
# max_df removes all words that are too frequent in the dataset 
# min_df removes all words that occur only once in the dataset 
# using sklearn function CountVectorizer() and including parameter ngram_range=(1,4) to create a vocabulary of phrases from 1 to 4 words.
N_Gram_vectorizer16 = CountVectorizer(ngram_range=(1,4), max_df=0.35, min_df=2)
N_Gram_vectorizer20 = CountVectorizer(ngram_range=(1,4), max_df=0.35, min_df=2)
N_Gram_vectorizer1620 = CountVectorizer(ngram_range=(1,4), max_df=0.35, min_df=2)

# 2016 dataset
X_train_N_Gram16 = N_Gram_vectorizer16.fit_transform(X_train16)
X_test_N_Gram16 = N_Gram_vectorizer16.transform(X_test16)

# 2020 dataset
X_train_N_Gram20 = N_Gram_vectorizer20.fit_transform(X_train20)
X_test_N_Gram20 = N_Gram_vectorizer20.transform(X_test20)

# 2016-20 dataset
X_train_N_Gram1620 = N_Gram_vectorizer1620.fit_transform(X_train1620)
X_test_N_Gram1620 = N_Gram_vectorizer1620.transform(X_test1620)

###Logistic regression using BOW

How well does the training data predict the test data?

To create a logistic regression model we used sklearn machine learning built in logistic regression model (see imports) and tuned its hyperparameter C to find its optimal value. A theoretical description of the logistic regression method and its use case in the paper is discussed in section *4.1 Logistic regression using bag-of-words (BOW)*.

####Tunning hyperparameter C

Vizualization of varying hyperparameter C in logistic regression on train and test accuracies.
This topic is discussed in the paper in *section 5.1.2 Tuning hyperparameter C, regularization parameter.*


In [ ]:
#Vizualization of varying hyperparameter C in Logistic regression using Bag of words on train and test accuracies

acc_table_train16 = []
acc_table_test16 = []
acc_table_train20 = []
acc_table_test20 = []
acc_table_train1620 = []
acc_table_test1620 = []
# list of C values that the logistic regression models are tuned on
param_range= [0.001,0.01,0.05,0.1,0.4,0.6,0.8,1.0,1.2,1.4,1.6,1.8,2.0,2.5,3,3.5,4,5,6,7,8,9,10]

# creating 3 horizontally aligned subplots
fig, axs = plt.subplots(1, 3, sharey='row', figsize=(22.5,8))

for i in param_range:
  #Apply logistic regression model to training data and estimate the model through tuning  hyperparamter c with the param_range list given above
  logit_BoW_16 = LogisticRegression(C=i)
  logit_BoW_20 = LogisticRegression(C=i)
  logit_BoW_1620 = LogisticRegression(C=i)

  #Fits the model according to the given training data.
  logit_BoW_16.fit(X_train_BagOfWords16, y_train16) 
  logit_BoW_20.fit(X_train_BagOfWords20, y_train20) 
  logit_BoW_1620.fit(X_train_BagOfWords1620, y_train1620) 

  #Predict test set using logistic regression and a given C
  acc_table_test16.append(logit_BoW_16.score(X_test_BagOfWords16, y_test16))
  acc_table_train16.append(logit_BoW_16.score(X_train_BagOfWords16, y_train16))
  acc_table_test20.append(logit_BoW_20.score(X_test_BagOfWords20, y_test20))
  acc_table_train20.append(logit_BoW_20.score(X_train_BagOfWords20, y_train20))
  acc_table_test1620.append(logit_BoW_1620.score(X_test_BagOfWords1620, y_test1620))
  acc_table_train1620.append(logit_BoW_1620.score(X_train_BagOfWords1620, y_train1620))

#Vizualization of results
axs[0].plot(param_range, acc_table_train16, label="Train score", color="g")
axs[0].plot(param_range, acc_table_test16,  label="Test score", color="r")
axs[0].set_title('2016')
axs[1].plot(param_range, acc_table_train20, label="Train score", color="g")
axs[1].plot(param_range, acc_table_test20, label="Test score", color="r")
axs[1].set_title('2020')
axs[2].plot(param_range, acc_table_train1620, label="Train score", color="g")
axs[2].plot(param_range, acc_table_test1620, label="Test score", color="r")
axs[2].set_title('2016-20')
fig.suptitle('Accuracy of the logistic regression using BOW in all datasets')
axs[0].set(xlabel='C value', ylabel='Accuracy')
axs[1].set(xlabel='C value')
axs[2].set(xlabel='C value')
fig.subplots_adjust(wspace=0.05, hspace=0.05)
axs[0].grid()
axs[1].grid()
axs[2].grid()
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.2)

fig.show()

####Accuracy results

Accuracy of the logistic regression with BOW for all three datasets: 2016, 2020, 2016-20. These results are presented and discussed in the paper in section *5.1.3 Key results for bag-of-words and n-gram*.


In [ ]:
# With what accuracy does the logistic regression with BOW predict the political affiliation of tweets in the test sets for all three datasets: 2016, 2020, 2016-20

logit_BoW_16 = LogisticRegression(C=1)
logit_BoW_20 = LogisticRegression(C=1)
logit_BoW_1620 = LogisticRegression(C=1)

logit_BoW_16.fit(X_train_BagOfWords16, y_train16) 
logit_BoW_20.fit(X_train_BagOfWords20, y_train20) 
logit_BoW_1620.fit(X_train_BagOfWords1620, y_train1620) 

myList = [
    ["Year","Machine Learning method","Classifier","Testset accuracy C=1",],
    ["2016", "Bag of Words", "Logistic Regression", round(logit_BoW_16.score(X_test_BagOfWords16, y_test16)*100,3)],
    ["2020", "Bag of Words", "Logistic Regression", round(logit_BoW_20.score(X_test_BagOfWords20, y_test20)*100,3)],
    ["2016+2020", "Bag of Words", "Logistic Regression", round(logit_BoW_1620.score(X_test_BagOfWords1620, y_test1620)*100,3)]
]

TableIt.printTable(myList, useFieldNames=True)
print('')

###Logistic regression using N-Gram
A theoretical description of the logistic regression method and its use case in the paper is discussed in section *4.1 Logistic regression using bag-of-words (BOW)*.

####Tunning hyperparameter C

This topic is discussed in the paper in section *5.1.2 Tuning hyperparameter C, regularization parameter*

In [ ]:
#Vizualization of varying hyperparameter C in the N-Gram model on train and test accuracies

acc_table_train16 = []
acc_table_test16 = []
acc_table_train20 = []
acc_table_test20 = []
acc_table_train1620 = []
acc_table_test1620 = []
# list of C values that the logistic regression models are tuned on
param_range= [0.005,0.001,0.01,0.05,0.1,0.4,0.6,0.8,1.0,1.2,1.4,1.6,1.8,2.0,2.5,3,3.5,4,5,6,7,8,9,10]

fig, axs = plt.subplots(1, 3, sharey='row', figsize=(22.5,8))
for i in param_range:
  #Apply logistic regression model to training data
  logit_NG_16 = LogisticRegression(C=i)
  logit_NG_20 = LogisticRegression(C=i)
  logit_NG_1620 = LogisticRegression(C=i)

  #Fits the model according to the given training data.
  logit_NG_16.fit(X_train_N_Gram16, y_train16) 
  logit_NG_20.fit(X_train_N_Gram20, y_train20) 
  logit_NG_1620.fit(X_train_N_Gram1620, y_train1620) 

  #Predict test set using logistic regression and a given C
  acc_table_test16.append(logit_NG_16.score(X_test_N_Gram16, y_test16))
  acc_table_train16.append(logit_NG_16.score(X_train_N_Gram16, y_train16))
  acc_table_test20.append(logit_NG_20.score(X_test_N_Gram20, y_test20))
  acc_table_train20.append(logit_NG_20.score(X_train_N_Gram20, y_train20))
  acc_table_test1620.append(logit_NG_1620.score(X_test_N_Gram1620, y_test1620))
  acc_table_train1620.append(logit_NG_1620.score(X_train_N_Gram1620, y_train1620))

#Vizualization of results
axs[0].plot(param_range, acc_table_train16, label="Train score", color="g")
axs[0].plot(param_range, acc_table_test16, label="Test score", color="r")
axs[0].set_title('2016')
axs[1].plot(param_range, acc_table_train20, label="Train score", color="g")
axs[1].plot(param_range, acc_table_test20, label="Test score", color="r")
axs[1].set_title('2020')
axs[2].plot(param_range, acc_table_train1620, label="Train score", color="g")
axs[2].plot(param_range, acc_table_test1620, label="Test score", color="r")
axs[2].set_title('2016 - 20')
fig.suptitle('Accuracy of the logistic regression using n-gram in all datasets')
axs[0].set(xlabel='C value', ylabel='Accuracy')
axs[1].set(xlabel='C value')
axs[2].set(xlabel='C value')
fig.subplots_adjust(wspace=0.1, hspace=0.1)
axs[0].grid()
axs[1].grid()
axs[2].grid()
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

fig.show()

####Accuracy results

Accuracy of the logistic regression with N-Gram for all three datasets: 2016, 2020, 2016-20.
These results are presented and discussed in the paper in *section 5.1.3 Key results for bag-of-words and n-gram*.

In [ ]:
# How well does the N-Gram training data predict the test set for all three datasets: 2016, 2020, 2016-20

#Apply logistic regression model to training data
logit_NG_16 = LogisticRegression(C=1)
logit_NG_20 = LogisticRegression(C=1)
logit_NG_1620 = LogisticRegression(C=1)

#Fits the model according to the given training data.
logit_NG_16.fit(X_train_N_Gram16, y_train16) 
logit_NG_20.fit(X_train_N_Gram20, y_train20) 
logit_NG_1620.fit(X_train_N_Gram1620, y_train1620) 

myList = [
    ["Year","Machine Learning method","Classifier","Testset accuracy C=1"],
    ["2016", "N-Gram", "Logistic Regression", round(logit_NG_16.score(X_test_N_Gram16, y_test16)*100,3)],
    ["2020", "N-Gram", "Logistic Regression", round(logit_NG_20.score(X_test_N_Gram20, y_test20)*100,3)],
    ["2016+2020", "N-Gram", "Logistic Regression", round(logit_NG_1620.score(X_test_N_Gram1620, y_test1620)*100,3)]
]

TableIt.printTable(myList, useFieldNames=True)
print('')

###Cross validation on both models

We use a k-folds cross validation technique with 5 folds.

Steps of k-folds cross validation method:
*   Partition the data into a number of subsets
*   Hold out a set at a time and train the model on remaining set
*   Test model on hold out set
*   Repeat the process for each subset of the dataset

Cross validation is presented and discussed in the paper in section *5.1.1    Cross validation*.

In [ ]:
# Cross-validation is a statistical method that can be used to verify the level of overfitting in machine learning models.
# For computing cross validation we used sklearn cross_val_score() function that returns a list of k accuracies, one for each tested subset in the training data.
# Parameters
# Logistic regression: training a model with logistic regression and its hyperparameter set to 1
# X_train_BagOfWords16 and y_train16
# we use cross validation with 5 folds: cv=5, giving 80% training and 20% test data

# Cross validation on bag of words datasets
scores_BagOfWords16 = cross_val_score(LogisticRegression(C=1), X_train_BagOfWords16, y_train16, cv=5) # 2016 dataset
scores_BagOfWords20 = cross_val_score(LogisticRegression(C=1), X_train_BagOfWords20, y_train20, cv=5) # 2020 dataset
scores_BagOfWords1620 = cross_val_score(LogisticRegression(C=1), X_train_BagOfWords1620, y_train1620, cv=5) # 2016-20 dataset

# Cross validation on n-gram datasets
scores_N_Gram16 = cross_val_score(LogisticRegression(C=1), X_train_N_Gram16, y_train16, cv=5) # 2016 dataset
scores_N_Gram20 = cross_val_score(LogisticRegression(C=1), X_train_N_Gram20, y_train20, cv=5) # 2020 dataset
scores_N_Gram1620 = cross_val_score(LogisticRegression(C=1), X_train_N_Gram1620, y_train1620, cv=5) # 2016-20 dataset

# Returns a table illustrating the cross validation scores of the training sets for both methods in 2016 and 2020.
myList = [
    ["Year      | Method", "Cross Validation scores", "Mean accuracy"],
    ["2016      | Bag-of-Words",scores_BagOfWords16,sum(scores_BagOfWords16)/len(scores_BagOfWords16)],
    ["          | N-Gram",scores_N_Gram16, sum(scores_N_Gram16)/len(scores_N_Gram16)],
    ["2020      | Bag-of-Words",scores_BagOfWords20, sum(scores_BagOfWords20)/len(scores_BagOfWords20)],
    ["          | N-Gram",scores_N_Gram20, sum(scores_N_Gram20)/len(scores_N_Gram20)],
    ["2016+2020 | Bag-of-Words",scores_BagOfWords1620, sum(scores_BagOfWords1620)/len(scores_BagOfWords1620)],
    ["          | N-Gram",scores_N_Gram1620 , sum(scores_N_Gram1620)/len(scores_N_Gram1620)]
]
TableIt.printTable(myList, useFieldNames=True)

###Confusion Matrix

By definition a confusion matrix C is such that Cij is equal to the number of observations known to be in group i and predicted to be in group j.

Thus in our binary classification, the count of true Republican tweets is C[0][0] and false Republican tweets is C[1][0]. Likewise, the count of true Democrat tweets is C[0][1], and false Democrat tweets C[1][1].

These results are analyzed in section 5.1.3 Key results for bag-of-words and n-gram.

In [ ]:
pred_lr_bow16 = logit_BoW_16.predict(X_test_BagOfWords16)
pred_lr_bow20 = logit_BoW_20.predict(X_test_BagOfWords20)
pred_lr_bow1620 = logit_BoW_1620.predict(X_test_BagOfWords1620)

pred_lr_ng16 = logit_NG_16.predict(X_test_N_Gram16)
pred_lr_ng20 = logit_NG_20.predict(X_test_N_Gram20)
pred_lr_ng1620 = logit_NG_1620.predict(X_test_N_Gram1620)


confB16 = confusion_matrix(y_test16, pred_lr_bow16)
confB20 = confusion_matrix(y_test20, pred_lr_bow20)
confB1620 = confusion_matrix(y_test1620, pred_lr_bow1620)
confNG16 = confusion_matrix(y_test16, pred_lr_ng16)
confNG20 = confusion_matrix(y_test20, pred_lr_ng20)
confNG1620 = confusion_matrix(y_test1620, pred_lr_ng1620)


print("Confusion matrix")
myList = [
    ["Year","ML method","Classifier","True Democrat", "False Republican","False Democrat", "True Republican"],
    ["2016", "Bag of Words", "Log Reg", confB16[0][0], confB16[0][1], confB16[1][0], confB16[1][1]],
    ["2016", "N-Gram", "Log Reg", confNG16[0][0], confNG16[0][1], confNG16[1][0], confNG16[1][1]],
    ["2020", "Bag of Words", "Log Reg", confB20[0][0], confB20[0][1], confB20[1][0], confB20[1][1]],
    ["2020", "N-Gram", "Log Reg", confNG20[0][0], confNG20[0][1], confNG20[1][0], confNG20[1][1]],
    ["2016+2020", "Bag of Words", "Log Reg", confB1620[0][0], confB1620[0][1], confB1620[1][0], confB1620[1][1]],
    ["2016+2020", "N-Gram", "Log Reg", confNG1620[0][0], confNG1620[0][1], confNG1620[1][0], confNG1620[1][1]]
]

TableIt.printTable(myList, useFieldNames=True)
